In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np

In [2]:
data = pd.read_csv("content_topic_prediction/Topic_prediction_data_train.txt",sep='\t',header=None)
data.columns = ['text','uid','type','timestamp']
labels = pd.read_csv("content_topic_prediction/Topic_prediction_labels_train.txt",sep='\t',header=None)
labels.columns = ['label']
data['label'] = labels['label']



In [3]:
data.head()

,text,uid,type,timestamp,label
0,distinct consubstanti translat roman missal pu...,0,Q,0.01,9
1,cathol christian power bind loos catholic beli...,0,Q,0.01,0
2,read scriptur common worship christian church ...,0,Q,0.01,9
3,accord person polit libertarian compat golden ...,0,Q,0.01,0
4,criteria cathol determin testament charact pre...,0,Q,0.01,0


In [4]:
data.shape

(98116, 5)

In [5]:
data = data[(data['text'].notnull())]

In [6]:
data.shape

(96269, 5)

In [7]:
#convert text to bag of words
corpus = data['text'].values
v = CountVectorizer()
bow = pd.DataFrame(v.fit_transform(corpus).todense())
bow['label'] = data['label'].astype(int).astype(str)
bow = bow[bow['label'].notnull()]
inputs = bow[bow.columns.tolist()[:-1]]
labels = bow['label']
del(bow)

In [8]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000)

In [9]:
train_data_features = vectorizer.fit_transform(data["text"])
features= np.array(train_data_features)

In [11]:
lr = LogisticRegression(random_state=1)
lr_params = {'penalty':('l1','l2'),'C':[0.1,1,10]}
lr_cv = GridSearchCV(lr,lr_params,cv=2)
nb = MultinomialNB()
nb_params = {'alpha':[0.001,0.5,1]}
nb_cv = GridSearchCV(nb,nb_params,cv=2)
svc = LinearSVC(random_state=1)
svc_params = {'penalty':('l1','l2'),'C':[0.1,1,10]}
svc_cv = GridSearchCV(lr,lr_params,cv=2)

lr_cv.fit(train_data_features,data["label"])
nb_cv.fit(train_data_features,data["label"])
svc_cv.fit(train_data_features,data["label"])
print(lr_cv.best_params_)
print(nb_cv.best_params_)
print(svc_cv.best_params_)

/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


{'C': 1, 'penalty': 'l2'}
{'alpha': 0}
{'C': 1, 'penalty': 'l2'}


In [18]:
classifier = VotingClassifier(estimators=[('lr',LogisticRegression(random_state=1,C=1,penalty='l2')),('nb',MultinomialNB(alpha=0.001)),('svc',LinearSVC(random_state=1,C=1,penalty='l2'))],voting='hard')
scores = cross_val_score(classifier, train_data_features, data["label"], cv=3, scoring='accuracy')

/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
scores.mean()

0.7506195761113652